In [1]:
!pip install  -U -q trl peft bitsandbytes wandb
# Tested with transformers==4.47.1, trl==0.14.0, datasets==3.2.0, peft==0.14.0, accelerate==1.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import wandb

wandb.login()

# Name the project
%env WANDB_PROJECT=Cosmos-8B-Instruct-GRPO

env: WANDB_PROJECT=Cosmos-8B-Instruct-GRPO


In [5]:
from datasets import load_dataset

dataset_id = "alibayram/turkish_mmlu"
train_dataset = load_dataset(dataset_id, split=["train"])
train_dataset = train_dataset[0]
train_dataset = train_dataset.remove_columns(["bolum","konu","aciklama","__index_level_0__"])
train_dataset = train_dataset.shuffle(seed=42).select(range(6600))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.40k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/54.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/279341 [00:00<?, ? examples/s]

In [6]:
# Test dataset
print(train_dataset[0])

{'soru': 'Aşağıdakilerden hangisi medyanın işlevlerinden biri değildir?', 'cevap': 2, 'secenekler': ['Çevre gözetimi', 'İlişkilendirme', 'Kişisel gelişime destek vermek', 'Kültür aktarmak', 'Eğlendirmek']}


In [7]:
SYSTEM_PROMPT = (
    "Kullanıcı ve asistan arasında bir konuşma. Kullanıcı bir soru soruyor, asistan bu soruyu çözüyor ve cevap veriyor. Asistan "
    "soruyu öncelikle kendi zihininde düşünüyor ve sonra kullanıcıya yanıt veriyor. Akıl yürütme "
    "işlemi <düşünce> </düşünce>, cevaplar <cevap> </cevap> etiketleri arasında gösteriliyor."
    "Örneğin: <düşünce>Akıl yürütme kısmı</düşünce><cevap>Cevap Kısmı</cevap>"
)


def make_conversation(example):
    secenekler = [a + b for a,b in zip(["A)","B)","C)","D)","E)"],example["secenekler"])]
    user_content = example["soru"] + " Seçenekler: " + " ".join(secenekler)
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_content},
        ],
    }

train_dataset = train_dataset.map(make_conversation)
train_dataset = train_dataset.remove_columns(["secenekler","soru"])

Map:   0%|          | 0/6600 [00:00<?, ? examples/s]

In [8]:
print(train_dataset)

Dataset({
    features: ['cevap', 'prompt'],
    num_rows: 6600
})


In [26]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_id = "ytu-ce-cosmos/Turkish-Llama-8b-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype="auto",
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

1

In [27]:
model.resize_token_embeddings(len(tokenizer))
print("Pad token added.")

Pad token added.


In [28]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=64,
    lora_alpha=64,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 27,262,976 || all params: 8,057,532,416 || trainable%: 0.3384


In [29]:
import re


def format_reward(completions, **kwargs):
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<düşünce>.*?</düşünce>\s*<cevap>.*?</cevap>$"
    completion_contents = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, content) for content in completion_contents]
    return [1.0 if match else 0.0 for match in matches]

In [30]:
def accuracy_reward(completions, **kwargs):
    """Reward function that checks if the completion is the same as the ground truth."""
    solutions = kwargs["cevap"]
    completion_contents = [completion[0]["content"] for completion in completions]
    rewards = []
    for content, solution in zip(completion_contents, solutions):
        correct_letter = list(["A","B","C","D","E"])[solution]
        # Regex to find standalone option letters (A-E) followed by optional ')' and punctuation/whitespace
        option_pattern = re.compile(r'\b([A-E])\)?(?=\s|\.|,|$)', re.IGNORECASE)
        matches = option_pattern.findall(content.upper())
        letters_found = [m.upper() for m in matches]

        # Check if correct_letter is present and no incorrect letters are mentioned
        if correct_letter in letters_found and all(letter == correct_letter for letter in letters_found):
            rewards.append(1.0)  # Full reward for correct answer and no other options
        else:
            rewards.append(0.0)  # No reward otherwise
    return rewards

In [31]:
from trl import GRPOConfig

# Configure training arguments using GRPOConfig
training_args = GRPOConfig(
    output_dir="Cosmos-8B-GRPO",
    learning_rate=1e-5,
    remove_unused_columns=False,  # to access the solution column in accuracy_reward
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    bf16=True,
    # Parameters that control de data preprocessing
    max_completion_length=64,  # default: 256
    num_generations=4,  # default: 8
    max_prompt_length=128,  # default: 512
    # Parameters related to reporting and saving
    report_to="wandb",
    use_vllm=True,
    logging_steps=1,
    push_to_hub=True,
    save_strategy="steps",
    save_steps=10,
)

In [32]:
print(accuracy_reward([[{"content":"Answer is A"}]],cevap=[0]))

[1.0]


In [33]:
from trl import GRPOTrainer

trainer = GRPOTrainer(
    model=model, reward_funcs=[format_reward, accuracy_reward],  args=training_args, train_dataset=train_dataset,
    processing_class=tokenizer,
)

In [38]:
trainer.train()
wandb.finish()

Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000


KeyboardInterrupt: 

In [ ]:
trainer.save_model(training_args.output_dir)
trainer.push_to_hub(dataset_name=dataset_id)